## Loading data into neo4j


In [5]:
# from neo4j import GraphDatabase

# class Neo4jConnect:
#     def __init__(self, user, password):
#         self.driver = GraphDatabase.driver("bolt://localhost:7687", auth=(user, password))
#         self.user = user
#     def close(self):
#         self.driver.close()
        
#     def query(self, query):
#         session = self.driver.session(database="cran")
#         result = session.run(query)
#         return result
    
#     def clean_database(self):
#         """
#         Delete all nodes and edge in Neo4j test container
#         """
#         q = "MATCH (n) DETACH DELETE (n)"
#         try:
#             session = self.driver.session()
#             session.run(q)
#         except Exception as e:
#             print(e)
    
#     def test_neo4j_connection(self):
#         """Test neo4j connection
#         """
#         session = self.driver.session()
#         result = session.run("Match () Return 1 Limit 1")
#         return result
#     def __str__(self):
#         return f'Connection successful for user: {self.user}.\nConnected to Neo4J database uri: {self.uri}'
    
# connection = Neo4jConnect('admin', 'cran2graph')

In [1]:
## Setting up connections

from graphutils.utils import Neo4jConnect

connection = Neo4jConnect('admin', 'cran2graph')

In [5]:
# Testing the connection to neo4j
test_query = """
MATCH (a:Person {person: 'Jing Zhang'})-[:MAINTAINS]->(p:Package)
RETURN a,p
LIMIT 10;
"""

package_maintainer = connection.query(test_query).data()
package_maintainer

[{'a': {'person': 'Jing Zhang'},
  'p': {'license': 'GPL-3',
   'package': 'do',
   'md5sum': 'f89af897b6b03444fdfa63d91d51ef0a',
   'description': 'Flexibly convert data between long and wide format using just two\n    functions: reshape_toLong() and reshape_toWide().',
   'published': '2021-08-03',
   'version': '2.0.0.0'}},
 {'a': {'person': 'Jing Zhang'},
  'p': {'license': 'GPL-3',
   'package': 'nomogramFormula',
   'md5sum': '552ddb326bbff202be3fe31ec2f36ff3',
   'description': '\n  A nomogram, which can be carried out in rms package, provides a \n    graphical explanation of a prediction process. However, it is not very easy\n    to draw straight lines, read points and probabilities accurately. Even, it \n    is hard for users to calculate total points and probabilities for all \n    subjects.\n  This package provides formula_rd() and formula_lp() functions to fit the \n    formula of total points with raw data and linear predictors respectively by\n    polynomial regression. F

### Create constraint on the various nodes

In [22]:
# Create a constraint on 'Package' nodes based on the 'package' property
constraint_package_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Package) REQUIRE p.package IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_person_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Person) REQUIRE a.person IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_license_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (l:License) REQUIRE l.license IS UNIQUE"

# # Create a constraint on 'Person' nodes based on the 'person' property
# constraint_license_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (l:License) REQUIRE l.license IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_institution_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (i:Institution) REQUIRE i.institution IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_imports_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (m:Import) REQUIRE m.package IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_dependency_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (d:Dependency) REQUIRE d.name IS UNIQUE"

# Create an index on the 'person' property of 'Person' nodes
person_index_query = "CREATE INDEX ON :Person(person)"

constraints = [constraint_dependency_query, constraint_imports_query, constraint_license_query, constraint_institution_query, constraint_person_query, constraint_package_query]

# for constraint in constraints:
#     print(f"Creating the constraints with: {constraint}")
#     connection.query(constraint)
# connection.close()


#db_session = establish_connection()

for constraint in constraints:
    print(f"Creating the constraints with: {constraint}")
    connection.query(constraint)
#db_session.close()
    

Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (d:Dependency) REQUIRE d.name IS UNIQUE


C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\808217691.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database="cran")


Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (m:Import) REQUIRE m.package IS UNIQUE
Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (l:License) REQUIRE l.license IS UNIQUE
Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (i:Institution) REQUIRE i.institution IS UNIQUE
Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (a:Person) REQUIRE a.person IS UNIQUE
Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (p:Package) REQUIRE p.package IS UNIQUE


In [ ]:
## Loading package nodes from csv file
load_package_query = """
// Load 'Package' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
CALL {
    WITH row
    MERGE (p:Package {
        package: row.package,
        version: row.version,
        license: row.license,
        md5sum: row.md5sum,
        description: row.description,
        published: row.published_date
    })
    ON CREATE SET
        p.package = row.package,
        p.version = row.version,
        p.license = row.license,
        p.md5sum = row.md5sum,
        p.description = row.description,
        p.published = row.published_date
    ON MATCH SET
        p.package = row.package,
        p.version = row.version,
        p.license = row.license,
        p.md5sum = row.md5sum,
        p.description = row.description,
        p.published = row.published_date
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(load_package_query)
connection.close()
print(f"end operation....")

In [53]:
## THis fix the issue
## Loading package nodes from csv file
load_package_query = """
// Load 'Package' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
CALL {
    WITH row
    MERGE (p:Package {package: row.package})
    ON CREATE SET
        p.version = row.version,
        p.license = row.license,
        p.md5sum = row.md5sum,
        p.description = row.description,
        p.published = row.published_date
    ON MATCH SET
        p.version = row.version,
        p.license = row.license,
        p.md5sum = row.md5sum,
        p.description = row.description,
        p.published = row.published_date
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(load_package_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [33]:
## Loading package nodes from csv file
load_package_query = """
// Load 'Package' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
CALL {
    WITH row
    MERGE (p:Package {
        package: row.package,
        version: row.version,
        license: row.license,
        md5sum: row.md5sum,
        description: row.description,
        published: row.published_date
    })
    ON CREATE SET
        package = row.package,
        version = row.version,
        license = row.license,
        md5sum = row.md5sum,
        description = row.description
        published = row.published_date
    ON MATCH SET
        package = row.package,
        version = row.version,
        license = row.license,
        md5sum = row.md5sum,
        description = row.description
        published = row.published_date
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(load_package_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [6]:
## working comment
## Loading person nodes from csv file
load_person_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
CALL {
    WITH row
    MERGE (a:Person {
        person: COALESCE(row.author, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(load_person_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [23]:
## working comment
## Loading imports nodes from csv file
load_dependency_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
CALL {
    WITH row
    MERGE (a:Dependency {
        name: COALESCE(row.imports, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(load_dependency_query)
connection.close()
print(f"end operation....")

loading query....


C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\808217691.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database="cran")


end operation....


In [3]:
## THis fix the issue
## Loading package nodes from csv file
## Loading person nodes from csv file
load_person_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
CALL {
    WITH row
    MERGE (a:Person {
        person: COALESCE(row.author, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(load_person_query)
connection.close()
print(f"end operation....")

loading query....


SessionError: Session closed

In [11]:
## THis fix the issue
## Loading package nodes from csv file
## Loading person nodes from csv file
load_license_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
CALL {
    WITH row
    MERGE (l:License {
        license: COALESCE(row.license, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(load_license_query)
connection.close()
print(f"end operation....")

loading query....


C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\808217691.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database="cran")


end operation....


In [12]:
## THis fix the issue
## Loading package nodes from csv file
## Loading person nodes from csv file
load_institution_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
CALL {
    WITH row
    MERGE (i:Institution {
        institution: COALESCE(row.institution, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(load_institution_query)
connection.close()
print(f"end operation....")

loading query....


C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\808217691.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database="cran")


end operation....


In [ ]:
## THis fix the issue
## Loading package nodes from csv file
## Loading person nodes from csv file
load_institution_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
CALL {
    WITH row
    MERGE (i:Institution {
        institution: COALESCE(row.institution, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(load_institution_query)
connection.close()
print(f"end operation....")

In [7]:
## Loading contributed_to relationships
rel_contributed_to_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
MATCH (p:Package { package: row.package })
MATCH (a:Person { person: COALESCE(row.author, "N/A")})

MERGE (a)-[:CONTRIBUTED_TO]->(p);

"""
print(f"loading query....")
connection.query(rel_contributed_to_query)
connection.close()
print(f"end operation....")

loading query....


C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\808217691.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database="cran")


end operation....


In [8]:
## Loading contributed_to relationships
rel_maintains_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
MATCH (p:Package { package: row.package })
MATCH (a:Person { person: COALESCE(row.maintainer_name,"N/A")})

MERGE (a)-[:MAINTAINS]->(p);

"""
print(f"loading query....")
connection.query(rel_maintains_query)
connection.close()
print(f"end operation....")

loading query....


C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\808217691.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database="cran")


end operation....


In [16]:
## Loading contributed_to relationships
rel_license_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
MATCH (p:Package { package: row.package })
MATCH (l:License { license: COALESCE(row.license,"N/A")})

MERGE (p)-[:LICENSE_BY]->(l);

"""
print(f"loading query....")
connection.query(rel_license_query)
connection.close()
print(f"end operation....")

loading query....


C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\808217691.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database="cran")


end operation....


In [19]:
## Loading contributed_to relationships
rel_institution_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
MATCH (a:Person { person: COALESCE(row.maintainer_name,"N/A")})
MATCH (i:Institution { institution: COALESCE(row.institution ,"N/A")})

MERGE (a)-[:WORKS_IN]->(i);

"""
print(f"loading query....")
connection.query(rel_institution_query)
connection.close()
print(f"end operation....")

loading query....


C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\808217691.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database="cran")


end operation....


In [24]:
## Loading contributed_to relationships
rel_dependency_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_process_data.csv' AS row
MATCH (p:Package { package: row.package })
MATCH (d:Dependency { name: COALESCE(row.imports,"N/A")})

MERGE (p)-[:DEPENDS_ON]->(d);

"""
print(f"loading query....")
connection.query(rel_dependency_query)
connection.close()
print(f"end operation....")

loading query....


C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\808217691.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.driver.session(database="cran")


end operation....


In [30]:
import igraph
from neo4j import GraphDatabase

# Define a function to retrieve data from Neo4j
def get_package_data(tx):
    query = "MATCH (p:Package) RETURN p.package, p.description limit 25"
    result = tx.run(query)
    return result.data()

# Connect to your Neo4j database
uri = "neo4j://localhost:7687"
username = "admin"
password = "cran2graph"

# Create a Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))


# Retrieve package data from Neo4j
with driver.session(database="cran") as session:
    data = session.read_transaction(get_package_data)

# Preprocess the input text (e.g., remove stopwords and tokenize)
input_text = "package to perform data transformation and cleaning"

# Create a list of package descriptions
package_descriptions = [package["description"] for package in data]

C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\1262747138.py:21: DeprecationWarning: read_transaction has been renamed to execute_read
  data = session.read_transaction(get_package_data)


[{'p.package': 'A3', 'p.description': 'Supplies tools for tabulating and analyzing the results of predictive models. The methods employed are applicable to virtually any predictive model and make comparisons between different methodologies straightforward.'}, {'p.package': 'AalenJohansen', 'p.description': 'Provides the conditional Nelson-Aalen and Aalen-Johansen estimators. The methods are based on Bladt & Furrer (2023), in preparation.'}, {'p.package': 'AATtools', 'p.description': 'Compute approach bias scores using different scoring algorithms,\n    compute bootstrapped and exact split-half reliability estimates,\n    and compute confidence intervals for individual participant scores.'}, {'p.package': 'ABACUS', 'p.description': 'A set of Shiny apps for effective communication and understanding in statistics. The current version includes properties of normal distribution, properties of sampling distribution, one-sample z and t tests, two samples independent (unpaired) t test and anal